# First Team Task (Permutated MNIST)
## Loïc Rosset, Nanae Aubry, Kilian Ruchti, Lionel Ieri

## Import Libraries

In [ ]:
# Import module
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets

from copy import deepcopy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from model_task2c import PR_CNN

## Load dataset

In [ ]:
# load data
TRAIN_DATASET = "../dataset/png/mnist_permutated/train"
VALIDATION_DATASET = "../dataset/png/mnist_permutated/val"
TEST_DATASET = "../dataset/png/mnist_permutated/test"

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.ImageFolder(root=TRAIN_DATASET, transform=transform)
val_dataset = datasets.ImageFolder(root=VALIDATION_DATASET, transform=transform)
test_dataset = datasets.ImageFolder(root=TEST_DATASET, transform=transform)

params = {'batch_size': 64, 'shuffle': True}

train_loader = torch.utils.data.DataLoader(train_dataset, **params)
val_loader = torch.utils.data.DataLoader(val_dataset, **params)
test_loader = torch.utils.data.DataLoader(test_dataset, **params)

## 1) CNN

In [ ]:
def plot_graph(nb_epochs, train, val, step):
    plt.figure()
    plt.plot(np.arange(nb_epochs), train)
    plt.plot(np.arange(nb_epochs), val)
    plt.legend(['training', 'validation'])
    plt.xlabel('epoch')
    plt.ylabel(f'{step} value')
    plt.title(f'Train/val {step}');
        
    plt.show()

In [ ]:
class Training():
    
    def __init__(self, nb_epochs=20, device=torch.device('cpu')):
        self.nb_epochs = nb_epochs
        self.device = device
    
    def train(self, model, train_loader, optimizer, loss_func):
        model.train()
        
        losses = []
        correct_train_pred = 0
        
        for data, labels in train_loader:
            data = data.to(self.device)
            labels = labels.to(self.device)
            
            # Predict the classes of the model
            output = model(data)
        
            optimizer.zero_grad()
            
            # Compute the loss
            loss = loss_func(output, labels)
            
            # Perform backprop
            loss.backward()
            optimizer.step()
            
            # Save current loss
            losses.append(loss.item())
            
            # Save the number of correct classified items
            predicted_labels = output.argmax(dim=1)
            nb_correct = (predicted_labels == labels).sum().item()
            correct_train_pred += nb_correct
    
        train_accuracy = 100. * (correct_train_pred / len(train_loader.dataset))
        
        return np.mean(np.array(losses)), train_accuracy
    
    def validation(self, model, val_loader, loss_func):
        model.eval()
        
        losses = []
        correct_val_predictions = 0
        
        with torch.no_grad():
            for data, labels in val_loader:
                data = data.to(self.device)
                labels = labels.to(self.device)
                
                output = model(data)
                
                loss = loss_func(output, labels)
                
                # Save current loss
                losses.append(loss.item())
    
                # Save the number of correct classified items
                predicted_labels = output.argmax(dim=1)
                n_correct = (predicted_labels == labels).sum().item()
                correct_val_predictions += n_correct
                
        val_accuracy = 100. * (correct_val_predictions / len(val_loader.dataset))
                
        return np.mean(np.array(losses)), val_accuracy

    def _print_info(self, train_loss, val_loss, train_acc, val_acc):
        print(f'Train_loss: {train_loss:.3f} |\
                Val_loss: {val_loss:.3f} |\
                Train_acc: {train_acc:.3f} |\
                Val_acc: {val_acc:.3f}')

    def fit(self, model, train_loader, val_loader, optimizer, loss_func):
        train_losses, val_losses = [], []
        train_accuracies, val_accuracies = [], []

        for epoch in range(self.nb_epochs):
            train_loss, train_acc = self.train(model, train_loader, optimizer, loss_func)
            val_loss, val_acc = self.validation(model, val_loader, loss_func)

            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)

        self._print_info(train_losses[-1], val_losses[-1], train_accuracies[-1], val_accuracies[-1])

        return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
l_rate_set = [0.001, 0.01, 0.1]
nb_epochs = 5
trainer = Training(nb_epochs)

# best parameters: (validation acc, learning rate)
best_parameters = (float("-INF"), None, None)
# keep a copy of the best trained network
best_model = None

loss_function = nn.CrossEntropyLoss()

for l_rate in l_rate_set:
    cnn = PR_CNN()
    optimizer = torch.optim.SGD(cnn.parameters(), l_rate)
    
    # The training take place here :
    print(f'learning rate: {l_rate}')
    stats_training = trainer.fit(cnn, train_loader, val_loader, optimizer, loss_function)
    
    if stats_training[3][-1] > best_parameters[0]:
        best_parameters = (stats_training[3][-1], l_rate, stats_training)
        best_model = deepcopy(cnn)

best_acc, best_l_rate, best_stats = best_parameters
train_losses, val_losses, train_accuracies, val_accuracies = best_stats

print(f'\n\nBest parameters: {best_l_rate} learning_rate and validation accuracies: {best_acc:.2f}%')

plot_graph(nb_epochs, train_losses, val_losses, "Loss")
plot_graph(nb_epochs, train_accuracies, val_accuracies, "Accuracy")

### Accuracy

In [ ]:
loss_test, acc_test = trainer.validation(best_model, test_loader, loss_function)
print(f'Accuracy on the test dataset {acc_test:.2f}%')

## 2) MLP

## 3) Comparison with normal MNIST (results from 2b and 2c)